In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
flight = pd.read_excel("flight_unfucked.xlsx")
test = pd.read_excel("test_unfucked.xlsx")

flight.drop("Unnamed: 0", inplace = True, axis = 1)
test.drop("Unnamed: 0", inplace = True, axis = 1)

In [3]:
dummy = flight.copy(deep = True)

- kamus_gender = {"M" : 0, "F" : 1, "None" : None}
- kamus_trip = {"trip" : 0, "round": 1, "roundtrip" : 2}
- kamus_service = {"ECONOMY" : 0, "BUSINESS": 1}
- kamus_tx = {"NO" : 0, "YES" : 1}
- kamus_airlines = {'33199710eb822fbcfd0dc793f4788d30': 0,
                  '0a102015e48c1f68e121acc99fca9a05': 1,
                  '6c483c0812c96f8ec43bb0ff76eaf716': 2,
                  'ad5bef60d81ea077018f4d50b813153a': 3,
                  'e35de6a36d385711a660c72c0286154a': 4,
                  '74c5549aa99d55280a896ea50068a211': 5,
                  '9855a1d3de1c46526dde37c5d6fb758c': 6,
                  '6872b49542519aea7ae146e23fab5c08': 7}

In [4]:
#changing visited_city and log trans into usable
temp = []
for value in dummy["log_transaction"]:
    a = value.split(',')
    to_float = lambda x : [float(p) for p in x]
    a = to_float(a)
    temp.append(a)
dummy["log_transaction"] = temp


temp = []
for value in dummy["visited_city"]:
    a = value.split(',')
    temp.append(a)
dummy["visited_city"] = temp



In [5]:
temp = []
for value in test["log_transaction"]:
    a = value.split(',')
    to_float = lambda x : [float(p) for p in x]
    a = to_float(a)
    temp.append(a)
test["log_transaction"] = temp



temp = []
for value in test["visited_city"]:
    a = value.split(',')
    temp.append(a)
test["visited_city"] = temp

Tipe data tiap kolom

- account_id identifier
- member duration days = nominal, menunjukkan jumlah
- gender non weighted binary
- trip nominal
- service class = ordinal, binary
- price = continue
- is tx promo = binary
- no_of_seats = nominal jumlah
- airlines_names = nominal
- hotel_id = parse ke binary
- visited_city = list string
- log_trans = list float

## ide feature

- berapa banyak kota yang dikunjungi?
- kota mulai?
- kota akhir?
- melalui kota x?
- apakah mengunjungi lebih dari n kota?
- mean log_trans
- repeat customer? (dari log_trans)
- is alone? (seat number)

In [6]:
#jumlah kota dikunjungi
temp = []
start = []
end = []
num_transit= []
for i in dummy["visited_city"]:
    temp.append(len(i))
    start.append(i[0])
    end.append(i[-1])
    num_transit.append(len(i)-2)

num_trans = []
mean_trans = []
for i in dummy["log_transaction"]:
    num_trans.append(len(i))
    mean_trans.append(sum(i)/len(i))
    
is_alone = []

for i in dummy["no_of_seats"]:
    if i > 1:
        is_alone.append(0)
    else:
        is_alone.append(1)

dummy["is_alone"] = is_alone    
dummy["num_visited"] = temp
dummy["start"]= start
dummy["end"] = end
dummy["num_trans"] = num_trans
dummy["mean_trans"] = mean_trans
dummy["num_transit"] = num_transit

In [ ]:
sns.pairplot(dummy.drop("account_id", axis =1))

In [64]:
dummy = pd.concat([dummy, pd.get_dummies(dummy["start"], prefix = "start")],axis = 1)
dummy = pd.concat([dummy, pd.get_dummies(dummy["end"], prefix = "end")],axis = 1)
dummy = pd.concat([dummy, pd.get_dummies(dummy["airlines_name"].astype(str), prefix = "airline")],axis = 1)
dummy = pd.concat([dummy, pd.get_dummies(dummy["trip"].astype(str), prefix = "trip")] ,axis = 1)
dummy = pd.concat([dummy, pd.get_dummies(dummy["gender"].astype(str), prefix = "gender")],axis = 1)

In [65]:
dummy.head()

,account_id,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,hotel_id,...,airline_4,airline_5,airline_6,airline_7,trip_0,trip_1,trip_2,gender_0.0,gender_1.0,gender_nan
0,0,566,0.0,0,0,885898.00,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
1,1,607,0.0,0,0,2139751.25,0,2,1,0,...,0,0,0,0,1,0,0,1,0,0
2,2,648,1.0,0,0,2695550.00,0,1,1,0,...,0,0,0,0,1,0,0,0,1,0
3,3,418,1.0,0,0,1146665.00,0,1,1,0,...,0,0,0,0,1,0,0,0,1,0
4,4,537,1.0,0,0,1131032.50,0,1,2,0,...,0,0,0,0,1,0,0,0,1,0


In [81]:
dummy.corr()["hotel_id"]

account_id             -0.003923
member_duration_days    0.032732
gender                  0.013790
trip                    0.031024
service_class          -0.000057
price                   0.017619
is_tx_promo             0.100512
no_of_seats             0.025740
airlines_name           0.019342
hotel_id                1.000000
is_alone               -0.042015
num_visited             0.007527
num_trans              -0.035251
mean_trans              0.007860
num_transit             0.007527
start_Bali              0.002112
start_Jakarta          -0.012335
start_Jogjakarta        0.000528
start_Manado           -0.006197
start_Medan             0.016363
start_Semarang          0.003146
start_Surabaya         -0.001229
end_Aceh               -0.001229
end_Bali               -0.018603
end_Jakarta             0.012804
end_Medan               0.002222
end_Semarang            0.009456
airline_0              -0.027638
airline_1               0.003694
airline_2               0.006315
airline_3 

# DON'T FUCKING RUN THE CODE BELOW OR ELSE YOU WILL SUFFER LAG AND SHIT

Kode di bawah nyoba train dan test semua fitur menggunakan Oversampling agar bisa... ummm... lebih imbang datanya, hasil : bisa training prediksi 0 dan 1 gak cuman 0 doang, tapi akurasi menurun drastis

In [111]:
pp= ['account_id','gender', 'trip','airlines_name','start',
       'end', "log_transaction", "visited_city"]
ppp = ['account_id','gender', 'trip','airlines_name','start',
       'end', "log_transaction", "visited_city", "hotel_id"]

In [ ]:
sement = dummy.drop(pp, axis = 1)

In [103]:
for i in range (9):    
    sement = pd.concat([sement, sement[sement["hotel_id"]==1]])
    sement.reset_index(drop = True, inplace = True)

In [104]:
sement = (sement - sement.min()) / (sement.max()-sement.min())

In [105]:
sement.columns

Index(['member_duration_days', 'service_class', 'price', 'is_tx_promo',
       'no_of_seats', 'hotel_id', 'is_alone', 'num_visited', 'num_trans',
       'mean_trans', 'num_transit', 'start_Bali', 'start_Jakarta',
       'start_Jogjakarta', 'start_Manado', 'start_Medan', 'start_Semarang',
       'start_Surabaya', 'end_Aceh', 'end_Bali', 'end_Jakarta', 'end_Medan',
       'end_Semarang', 'airline_0', 'airline_1', 'airline_2', 'airline_3',
       'airline_4', 'airline_5', 'airline_6', 'airline_7', 'trip_0', 'trip_1',
       'trip_2', 'gender_0.0', 'gender_1.0', 'gender_nan'],
      dtype='object')

In [106]:
sement.head()

,member_duration_days,service_class,price,is_tx_promo,no_of_seats,hotel_id,is_alone,num_visited,num_trans,mean_trans,...,airline_4,airline_5,airline_6,airline_7,trip_0,trip_1,trip_2,gender_0.0,gender_1.0,gender_nan
0,0.099122,0.0,0.008169,0.0,0.0,0.0,1.0,0.5,0.000610,0.030927,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.116269,0.0,0.029555,0.0,0.1,0.0,0.0,0.0,0.330689,0.108552,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,0.133417,0.0,0.039035,0.0,0.0,0.0,1.0,0.5,0.000915,0.197114,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.037223,0.0,0.012617,0.0,0.0,0.0,1.0,0.5,0.000305,0.123375,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,0.086993,0.0,0.012350,0.0,0.0,0.0,1.0,1.0,0.047285,0.190999,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [85]:
from sklearn.ensemble import AdaBoostClassifier as ada

In [ ]:
adaB =  ada(n_estimators = 100, random_state = 1, learning_rate = 0.01)
adaB.fit(sement.drop("hotel_id", axis = 1), sement["hotel_id"])

In [ ]:
tes =adaB.predict(dummy.drop(ppp,axis = 1))

In [99]:
set(tes)

,account_id,member_duration_days,gender,trip,service_class,price,is_tx_promo,no_of_seats,airlines_name,hotel_id,...,airline_4,airline_5,airline_6,airline_7,trip_0,trip_1,trip_2,gender_0.0,gender_1.0,gender_nan
23,23,891,0.0,0,0,1909362.0,0,2,4,1,...,1,0,0,0,1,0,0,1,0,0
28,27,1742,1.0,0,0,2760277.0,1,1,2,1,...,0,0,0,0,1,0,0,0,1,0
43,42,635,0.0,2,0,2197507.0,1,1,1,1,...,0,0,0,0,0,0,1,1,0,0
45,44,1600,0.0,0,0,2751320.0,0,2,2,1,...,0,0,0,0,1,0,0,1,0,0
87,85,2206,1.0,0,0,1363830.0,1,1,4,1,...,1,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117795,2512,670,1.0,2,0,2830496.5,1,1,1,1,...,0,0,0,0,0,0,1,0,1,0
117851,9219,396,0.0,0,0,1375727.6,1,1,1,1,...,0,0,0,0,1,0,0,1,0,0
117854,74748,627,1.0,2,0,1335100.0,0,1,3,1,...,0,0,0,0,0,0,1,0,1,0
117890,1044,1897,0.0,0,0,1602474.9,1,1,2,1,...,0,0,0,0,1,0,0,1,0,0
